# day 239,day 241

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests,os,zipfile
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
from sklearn.metrics import classification_report,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

# download the data

In [2]:
(train_data,test_data),meta_data = tfds.load(name='food101',
                                             split=['train','validation'],
                                             download=True,
                                             as_supervised=True,
                                             with_info=True,
                                             shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteOUPZ1Q/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteOUPZ1Q/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [ ]:
meta_data.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
})

In [3]:
# get the class names
class_names = meta_data.features['label'].names

# preprocess the data, batchify it and so on



In [4]:
# goal:
"""
1. create a preprocess function that will resize and recast the image
2. map the proprocess function train data and use the parallel computing units to speed up the process
3. shuffle the train data and batchify it by 32 and prefetch the next data to be processed
4. repeat the processes for test data except shuffling.
5. use AUTOTUNE method to let tensorflow choose the best buffer size
"""

# preprocess function
def preprocess(image,label,image_size=[224,224]):
  # resize the image
  image = tf.image.resize(image,image_size)

  # cast the image to tf.float32
  image = tf.cast(image,tf.float32)

  return image,label

# preparing the train data:
train_data = train_data.map(preprocess,num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# preparing the test data:
test_data = test_data.map(preprocess,num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# setting the callbacks

In [5]:
# setting the callbacks
def tensorboard(experiment_name):
  path = 'tensorboard/' + experiment_name

  c = tf.keras.callbacks.TensorBoard(log_dir=path)

  return c

def ModelCheckpoint(experiment_name):
  path = 'SavedModels/' + experiment_name

  c = tf.keras.callbacks.ModelCheckpoint(filepath=path,
                                         monitor='val_loss',
                                         save_weights_only=True,
                                         save_best_only=True,
                                         )

  return c


def early_stopping():
  c = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=3)
  return c

# use mixed precision and create the feature extractor

In [6]:
# goal:
"""
1. use tf.keras.mixed_precision.set_global_policy('mixed_float16')
2. build the feature extractor model
3. in the output layer set dtype to be equal to tf.float32 for good precision with loss function and optimizers

"""

# mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')


# download the base
base = tf.keras.applications.efficientnet.EfficientNetB4(include_top=False)
base.trainable=False

# create input layer
input_layer = tf.keras.Input(shape=[224,224,3],name='input_layer')

# put the input layer inside the base model
x = base(input_layer,training=False)

# put the above layer inside GlobalAveragePooling2D layer
x = tf.keras.layers.GlobalAveragePooling2D(name='GlobalAveragePooling2D')(x)

# put the above layer inside the output layer
output_layer = tf.keras.layers.Dense(units=len(class_names),activation='Softmax',dtype=tf.float32,name='output_layer')(x)

# create the model and name it model0
model0 = tf.keras.Model(input_layer,output_layer)


# compile the model
model0.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

# fit the model
tf.random.set_seed(42)
feature_extraction_epochs = 5
history0 = model0.fit(train_data,
          epochs=5,
           steps_per_epoch=len(train_data),
           validation_data=test_data,
           validation_steps=int(0.15*len(test_data)),
           callbacks=[tensorboard('model0'),ModelCheckpoint('model0')])

71686520/71686520 [==============================] - 0s 0us/step
Epoch 1/5
2368/2368 [==============================] - 275s 104ms/step - loss: 1.7861 - accuracy: 0.5577 - val_loss: 1.2590 - val_accuracy: 0.6674
Epoch 2/5
2368/2368 [==============================] - 257s 107ms/step - loss: 1.3085 - accuracy: 0.6576 - val_loss: 1.1581 - val_accuracy: 0.6933
Epoch 3/5
2368/2368 [==============================] - 250s 104ms/step - loss: 1.1627 - accuracy: 0.6919 - val_loss: 1.1080 - val_accuracy: 0.7029
Epoch 4/5
2368/2368 [==============================] - 252s 106ms/step - loss: 1.0643 - accuracy: 0.7181 - val_loss: 1.1028 - val_accuracy: 0.7076
Epoch 5/5
2368/2368 [==============================] - 266s 111ms/step - loss: 0.9886 - accuracy: 0.7358 - val_loss: 1.1033 - val_accuracy: 0.7031


In [7]:
# evaluate the model
model0.evaluate(test_data)

790/790 [==============================] - 75s 95ms/step - loss: 1.0929 - accuracy: 0.7027


[1.092885971069336, 0.7026930451393127]

# save the whole model

In [8]:
# save the whole model
save_path = Path('saved models')

# making a directory
save_path.mkdir(parents=True,exist_ok=True)

# save the model
model0.save(save_path)

In [9]:
# load the whole model

loaded_model = tf.keras.models.load_model(save_path)

In [ ]:
# chekcing the performance of the loaded_model against the original
loaded_model.evaluate(test_data)

790/790 [==============================] - 54s 66ms/step - loss: 0.9629 - accuracy: 0.7383


[0.9628571271896362, 0.7382574081420898]

# download someone else's model from internet and fine tune that

In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip

--2023-11-20 14:48:45--  https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.207, 108.177.125.207, 142.250.157.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976857 (16M) [application/zip]
Saving to: ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’

07_efficientnetb0_f 100%[===================>]  16.19M  11.3MB/s    in 1.4s    

2023-11-20 14:48:47 (11.3 MB/s) - ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’ saved [16976857/16976857]



In [ ]:
# unzip it

zip = zipfile.ZipFile('07_efficientnetb0_feature_extract_model_mixed_precision.zip')
zip.extractall()
zip.close()

In [ ]:
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

Archive:  07_efficientnetb0_feature_extract_model_mixed_precision.zip
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.data-00000-of-00001  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.index  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/saved_model.pb  
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/assets/


In [ ]:
# load and evaluate the downloaded_gs_model
file_path = 'downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision'
model1 = tf.keras.models.load_model(file_path)
model1.evaluate(test_data)

# the warnings are due to the fact that the saved model was from before tensorflow 2.5 version.
# actually the above code is working. look below!!

790/790 [==============================] - 55s 67ms/step - loss: 1.0881 - accuracy: 0.7066


[1.0880999565124512, 0.7066138386726379]

In [ ]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 softmax_float32 (Activatio  (None, 101)               0         
 n)                                                              
                                                             

In [ ]:
for layer in model0.layers:
  print(layer.dtype_policy)

<Policy "float32">
<Policy "mixed_float16">
<Policy "mixed_float16">
<Policy "float32">


# conclusion: our model model0 is far better!

# fine-tuning model0

In [10]:
# unfreeze all the layers
base.trainable = True

# refreeze the front layers
for layer in base.layers[:-8]:
  layer.trainable = False

# recompile the model with 10x lower learning_rate:
model0.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               metrics=['accuracy'])

# fit the model
model0.fit(train_data,
           epochs=100,
           steps_per_epoch=len(train_data),
           validation_data=test_data,
           validation_steps=int(0.15*len(test_data)),
           initial_epoch=feature_extraction_epochs,
           callbacks=[tensorboard('finetuned'),
                      ModelCheckpoint('finetuned'),
                      early_stopping()])

Epoch 6/100
2368/2368 [==============================] - 288s 114ms/step - loss: 0.8748 - accuracy: 0.7650 - val_loss: 1.0355 - val_accuracy: 0.7190
Epoch 7/100
2368/2368 [==============================] - 272s 114ms/step - loss: 0.7069 - accuracy: 0.8112 - val_loss: 1.0181 - val_accuracy: 0.7275
Epoch 8/100
2368/2368 [==============================] - 265s 111ms/step - loss: 0.5849 - accuracy: 0.8448 - val_loss: 0.9997 - val_accuracy: 0.7304
Epoch 9/100
2368/2368 [==============================] - 260s 109ms/step - loss: 0.4820 - accuracy: 0.8755 - val_loss: 1.0271 - val_accuracy: 0.7299
Epoch 10/100
2368/2368 [==============================] - 264s 111ms/step - loss: 0.3917 - accuracy: 0.9036 - val_loss: 1.0604 - val_accuracy: 0.7243
Epoch 11/100
2368/2368 [==============================] - 265s 111ms/step - loss: 0.3193 - accuracy: 0.9241 - val_loss: 1.0878 - val_accuracy: 0.7240


# save the model to reload later (after the system crashed!)

In [11]:
from pathlib import Path

# save the model
save_path = Path('saved_data')
save_path.mkdir(parents=True,exist_ok=True)

model0.save(save_path)

# load the model
loaded_model = tf.keras.models.load_model(save_path)



# evaluating the whole test data after fine tuning the model

In [ ]:
model0.evaluate(test_data)

790/790 [==============================] - 65s 82ms/step - loss: 0.8781 - accuracy: 0.7610


[0.8781448006629944, 0.7609900832176208]

In [12]:
model0.evaluate(test_data)

790/790 [==============================] - 72s 91ms/step - loss: 1.0799 - accuracy: 0.7270


[1.079860806465149, 0.7270098924636841]

In [60]:
# Download and evaluate fine-tuned model from Google Storag
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip

--2023-11-25 14:46:20--  https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.197.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46790356 (45M) [application/zip]
Saving to: ‘07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip’

07_efficientnetb0_f 100%[===================>]  44.62M   124MB/s    in 0.4s    

2023-11-25 14:46:21 (124 MB/s) - ‘07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip’ saved [46790356/46790356]



In [61]:
# Unzip fine-tuned model
!mkdir downloaded_fine_tuned_gs_model # create separate directory for fine-tuned model downloaded from Google Storage
!unzip 07_efficientnetb0_fine_tuned_101_classes_mixed_precision -d downloaded_fine_tuned_gs_model

Archive:  07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip
   creating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/
   creating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/variables/
  inflating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/variables/variables.data-00000-of-00001  
  inflating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/variables/variables.index  
  inflating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/saved_model.pb  
   creating: downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/assets/


In [63]:
load_downloaded_model = tf.keras.models.load_model('downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision')
load_downloaded_model

In [64]:
load_downloaded_model.evaluate(test_data)

790/790 [==============================] - 59s 68ms/step - loss: 0.9072 - accuracy: 0.8015


[0.9072170257568359, 0.8014653325080872]

# find precision,recall and f1-scores

# goals:
0.make predictions with the test_data and find its prediction labels.
1. create an empty list named actual classes to fetch and store the y_test.
2. get x_test and y_test from test_data by unbatching the test_data.
3. build the classification report
4. build the confusion matrix
5. find the wrong predictions with high accuracy.

In [65]:
# making predictions
predictions = load_downloaded_model.predict(test_data)

pred_classes = predictions.argmax(axis=1)

actual_classes= []

# unbatching the test_data to get y_test(acutal_class)
for x_test,y_test in test_data.unbatch():
  actual_classes.append(y_test.numpy())




790/790 [==============================] - 53s 65ms/step


In [66]:
pred_classes[:20]

array([65, 70, 34, 69, 23, 78, 13, 27, 27, 41, 32, 25, 48, 20, 53, 59, 51,
       38, 12, 34])

In [67]:
np.array(actual_classes)[:20]

array([37,  3, 73, 22, 59, 12, 39, 90,  6, 23, 44, 39,  6, 77, 56, 96, 42,
       96, 91, 25])

In [57]:
len(actual_classes)

25250

In [69]:
np.sum(np.array(actual_classes) == pred_classes)

240

In [34]:
len(pred_classes)

25250

In [51]:
max(actual_classes)

0

In [53]:
np.max(pred_classes)

100

In [70]:
#showing the classification report
print(classification_report(actual_classes,pred_classes))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       250
           1       0.02      0.02      0.02       250
           2       0.00      0.00      0.00       250
           3       0.00      0.00      0.00       250
           4       0.00      0.00      0.00       250
           5       0.00      0.00      0.00       250
           6       0.01      0.01      0.01       250
           7       0.00      0.00      0.00       250
           8       0.00      0.00      0.00       250
           9       0.01      0.01      0.01       250
          10       0.00      0.00      0.00       250
          11       0.01      0.01      0.01       250
          12       0.00      0.00      0.00       250
          13       0.02      0.02      0.02       250
          14       0.00      0.00      0.00       250
          15       0.00      0.00      0.00       250
          16       0.01      0.01      0.01       250
          17       0.01    

In [71]:
# building a confusion matrix
cm = confusion_matrix(actual_classes,pred_classes)
cm

array([[1, 2, 2, ..., 1, 1, 4],
       [0, 4, 4, ..., 0, 3, 2],
       [3, 1, 1, ..., 3, 2, 3],
       ...,
       [1, 3, 2, ..., 3, 1, 4],
       [3, 3, 1, ..., 4, 2, 1],
       [1, 3, 2, ..., 0, 4, 4]])

In [16]:
# plotting the confusion matrix;

plot_confusion_matrix(cm,
                      class_names=class_names,
                      figsize=(100,100))

plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [17]:
# getting the classification report in the form of dictionary
class_dic = classification_report(actual_classes,pred_classes,output_dict=True)

f1_score_dic = {}
# if the key is 'accuracy' stop the iteration. if not, then store the labels as keys and their corresponding values
for key,item in class_dic.items():
  if key == 'accuracy':
    break
  else:
    f1_score_dic[class_names[int(key)]] = np.round(class_dic[key]['f1-score'],4)

f1_score_dic

{'apple_pie': 0.017,
 'baby_back_ribs': 0.012,
 'baklava': 0.016,
 'beef_carpaccio': 0.0206,
 'beef_tartare': 0.0,
 'beet_salad': 0.0163,
 'beignets': 0.0196,
 'bibimbap': 0.004,
 'bread_pudding': 0.0122,
 'breakfast_burrito': 0.0,
 'bruschetta': 0.0133,
 'caesar_salad': 0.02,
 'cannoli': 0.0135,
 'caprese_salad': 0.0106,
 'carrot_cake': 0.0078,
 'ceviche': 0.014,
 'cheesecake': 0.0111,
 'cheese_plate': 0.0044,
 'chicken_curry': 0.0,
 'chicken_quesadilla': 0.0045,
 'chicken_wings': 0.0082,
 'chocolate_cake': 0.0,
 'chocolate_mousse': 0.0087,
 'churros': 0.0083,
 'clam_chowder': 0.0079,
 'club_sandwich': 0.0154,
 'crab_cakes': 0.0112,
 'creme_brulee': 0.0188,
 'croque_madame': 0.008,
 'cup_cakes': 0.0111,
 'deviled_eggs': 0.0081,
 'donuts': 0.0156,
 'dumplings': 0.0123,
 'edamame': 0.008,
 'eggs_benedict': 0.0157,
 'escargots': 0.0039,
 'falafel': 0.0181,
 'filet_mignon': 0.0156,
 'fish_and_chips': 0.0077,
 'foie_gras': 0.0144,
 'french_fries': 0.02,
 'french_onion_soup': 0.0038,
 'fren

In [18]:
# get the labels that are mostly wrong but high accuracy:
# goals:
'''
1. get pred_class
2. get actual_class
3. get pred_probs
4. filter the df in such a way that pred_class != actual_class and sort_values by pred_probs in desc order
'''

pred_probs = tf.reduce_max(predictions,axis=1).numpy()
df = pd.DataFrame({'actual_class':[class_names[int(x)] for x in actual_classes],
                   'pred_class':[class_names[int(x)] for x in pred_classes],
                   'pred_probs':pred_probs})

df

,actual_class,pred_class,pred_probs
0,ceviche,red_velvet_cake,0.838379
1,grilled_cheese_sandwich,baby_back_ribs,0.763672
2,cannoli,panna_cotta,0.638672
3,bibimbap,eggs_benedict,0.497070
4,edamame,churros,1.000000
...,...,...,...
25245,panna_cotta,bibimbap,0.999023
25246,lobster_roll_sandwich,carrot_cake,0.697754
25247,baby_back_ribs,french_onion_soup,1.000000
25248,gyoza,pad_thai,0.992676


In [19]:
# sorting the df to filter the most wrong predictions
boolean = df['actual_class'] != df['pred_class']

top_100_wrong = df[boolean].sort_values(by='pred_probs',ascending=False)[:100]

top_100_wrong

,actual_class,pred_class,pred_probs
22323,steak,lobster_roll_sandwich,1.0
1965,lasagna,edamame,1.0
9756,lasagna,mussels,1.0
20999,risotto,sashimi,1.0
23884,mussels,dumplings,1.0
...,...,...,...
18337,lobster_bisque,frozen_yogurt,1.0
9996,oysters,samosa,1.0
9993,guacamole,panna_cotta,1.0
23925,bread_pudding,miso_soup,1.0


# exploration,analysis and freethrows

TypeError: ignored

In [ ]:
actual_classes[0].dtype

dtype('int64')

In [ ]:
pred_classes[0].dtype

dtype('int64')

In [ ]:
len(list(f1_score_dic.keys()))

101

In [ ]:
len(pred_probs)

25250

In [ ]:
tf.reduce_max(predictions,axis=1).numpy()

array([1.    , 0.376 , 0.648 , ..., 0.9863, 0.9844, 0.9995], dtype=float16)

In [ ]:
test_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
class_dic = classification_report(actual_classes,pred_classes,output_dict=True)
class_dic['0']

{'precision': 0.004901960784313725,
 'recall': 0.004,
 'f1-score': 0.004405286343612335,
 'support': 250}

In [ ]:
class_dic['0']['f1-score']

0.004405286343612335

In [ ]:
np.round(class_dic['0']['f1-score'],4)

0.0044

In [ ]:
class_dic.items()

dict_items([('0', {'precision': 0.004901960784313725, 'recall': 0.004, 'f1-score': 0.004405286343612335, 'support': 250}), ('1', {'precision': 0.018867924528301886, 'recall': 0.02, 'f1-score': 0.019417475728155338, 'support': 250}), ('2', {'precision': 0.022222222222222223, 'recall': 0.024, 'f1-score': 0.023076923076923078, 'support': 250}), ('3', {'precision': 0.007905138339920948, 'recall': 0.008, 'f1-score': 0.007952286282306162, 'support': 250}), ('4', {'precision': 0.007017543859649123, 'recall': 0.008, 'f1-score': 0.007476635514018691, 'support': 250}), ('5', {'precision': 0.014134275618374558, 'recall': 0.016, 'f1-score': 0.0150093808630394, 'support': 250}), ('6', {'precision': 0.003745318352059925, 'recall': 0.004, 'f1-score': 0.003868471953578336, 'support': 250}), ('7', {'precision': 0.011152416356877323, 'recall': 0.012, 'f1-score': 0.011560693641618497, 'support': 250}), ('8', {'precision': 0.004464285714285714, 'recall': 0.004, 'f1-score': 0.004219409282700422, 'support':

# day 243

# EC1: Read up on learning rate scheduling and the learning rate scheduler callback. What is it? And how might it be helpful to this project?

* it can help to optimize our learning rate to an ideal value. so, we don't have to rely on that default 0.001 but custom tune the lr for our problem based on the requirements of the problem at hand.

# EC2: Read up on tensorflow dataloaders.

# summary of the learning:

# discussed optimization techniques so far:
1. **prefetching**: keeping the next batch of data ready while the current batch is being processed.
2. **batching**: batching allows data proprocessing little by little, easing the load for the computer.
3. **parallel computation**: parallel computation using num_parallel_calls uses more than one computing unit to process the data faster.

# undiscussed optimization techniques.


1. **caching**:  we have not used caching techniques with our dataset.

tf.data.Dataset.Cache allows for storage of the file opening and reading happen all at once and keep that information in cache memory so that the model doesn't have to spend so much time opening and reading the files at every epoch.

opening and reading happen all at once in the first epoch itself and the subsequent epochs take lesser amount of time but the caveat with the caching is if the cache memory of the system is not reasonably large enough to hold all those files at once then the system may crash!!!

2. **first batch it and then map it**: we have first mapped it and batched it, remember?

first batching and then mapping reduces the total time consumption.


3. **interleaving**: In TensorFlow, interleaving is a technique used to parallelize data loading during model training. It involves interleaving the elements of multiple datasets, allowing the model to be trained on data from different sources simultaneously. This can improve training performance by reducing the amount of time spent waiting for data to be loaded.

The interleave transformation is typically used in conjunction with the prefetch transformation. Prefetching fetches data from a remote location and stores it in a local cache before it is actually needed. This can further improve training performance by reducing the amount of time spent waiting for data to be transferred over the network.

The interleave transformation can be used to interleave data from multiple sources, such as:

* **Multiple files:** The interleave transformation can be used to interleave data from multiple files. This can be useful for training a model on a large dataset that is distributed across multiple files.
* **Multiple datasets:** The interleave transformation can be used to interleave data from multiple datasets. This can be useful for training a model on a variety of data sources, such as a dataset of images and a dataset of text.

The interleave transformation can be configured to interleave data in different ways, such as:

* **Sequential interleave:** Sequential interleave interleaves elements from one dataset until it is exhausted, then interleaves elements from the next dataset.
* **Random interleave:** Random interleave interleaves elements from all datasets randomly.

The best way to configure the interleave transformation depends on the specific application.

Here is an example of how to use the interleave transformation to interleave data from two files:

```python
dataset1 = tf.data.TextLineDataset("file1.txt")
dataset2 = tf.data.TextLineDataset("file2.txt")

interleaved_dataset = dataset1.interleave(dataset2, cycle_length=2, num_parallel_calls=tf.data.experimental.AUTOTUNE)
```

This code will interleave data from file1.txt and file2.txt. The cycle_length argument specifies that the transformation should interleave two elements from each dataset before moving on to the next dataset. The num_parallel_calls argument specifies that the transformation should be parallelized across multiple CPU threads.